<h3 style="text-align:center" style="color:rgb(0,0,255);">UNIVERSIDAD NACIONAL DE TRUJILLO</h3>
<h3 style="text-align:center" style="color:rgb(0,0,255);">FACULTAD DE CIENCIAS FÍSICAS Y MATEMÁTICAS</h3>
<h3 style="text-align:center" style="color:rgb(0,0,255);">ESCUELA PROFESIONAL DE INFORMÁTICA</h3>	
<img src="UNT.png" width="300" height="135" align="middle">
<h3 style="text-align:center" style="color:rgb(0,0,255);">PROYECTO:</h3>
<h3 style="text-align:center" style="color:rgb(0,0,255);">Identificación de Xanthomonas oryzae pv.
oryzicola (bacterial leaf blight of rice) en las hojas de arroz mediante visión por computador</h3>
<h3 style="text-align:center" style="color:rgb(0,0,255);">CURSO:</h3>
<h3 style="text-align:center" style="color:rgb(0,0,255);">Percepción y Visión por Computadoras</h3>
<h3 style="text-align:center" style="color:rgb(0,0,255);">ESTUDIANTES:</h3>
<h3 style="text-align:center" style="color:rgb(0,0,255);">-Mendez Cruz, Angely Yahayra</h3>
<h3 style="text-align:center" style="color:rgb(0,0,255);">-Mendez Cruz, Ciara Solange</h3>
<h3 style="text-align:center" style="color:rgb(0,0,255);">DOCENTE:</h3>
<h3 style="text-align:center" style="color:rgb(0,0,255);">Dra. Ing. Pedro Huaman Liz Sofia Raymunda</h3>
<h3 style="text-align:center" style="color:rgb(0,0,255);">Trujillo - Perú</h3>
<h3 style="text-align:center" style="color:rgb(0,0,255);">2023</h3>

<h1 style="color:RGB(0,0,255)"> 1. Instalación de librerías </h1>

In [1]:
!pip install scikit-learn
!pip install mahotas
!pip install scikit-image

<h1 style="color:RGB(0,0,255)"> 2. Importación de librerías </h1>

In [61]:
import os
import cv2
import numpy as np
#Segmentación
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
#Extracción de Características
import mahotas.features.texture as texture
from skimage.measure import label, regionprops
#Clasificador
from sklearn import svm
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
# Matriz de Confusión
from sklearn.metrics import confusion_matrix, classification_report
import pickle
#Grid Search
from sklearn.model_selection import train_test_split, GridSearchCV

<h1 style="color:RGB(0,0,255)"> 3. Adquisición de Imagen (Datasets: CABY Digital Library y Kaggle)</h1>

In [71]:
# Directorio de la dataset
dataset_dir = './data'

# Tamaño deseado para las imágenes
desired_size = (512, 512)

# Rutas de las carpetas de cada categoría
enfermas_dir = os.path.join(dataset_dir, 'enfermas')
sanas_dir = os.path.join(dataset_dir, 'sanas')
# otras_plagas_dir = os.path.join(dataset_dir, 'otras_plagas')

# Leer las imágenes de cada categoría
enfermas_images = []
for filename in os.listdir(enfermas_dir):
    if filename.endswith('.jpg'):
        image_path = os.path.join(enfermas_dir, filename)
        image = cv2.imread(image_path)
        image = cv2.resize(image, desired_size)
        enfermas_images.append(image)

sanas_images = []
for filename in os.listdir(sanas_dir):
    if filename.endswith('.jpg'):
        image_path = os.path.join(sanas_dir, filename)
        image = cv2.imread(image_path)
        image = cv2.resize(image, desired_size)        
        sanas_images.append(image)

<h1 style="color:RGB(0,0,255)"> 4. Preprocesamiento de imagen</h1>

In [72]:
# Mostrar Resultados 
# plt.figure(figsize=(13,5))

def preprocess_image(image):
           
#     plt.subplot(1,5,1)
#     plt.imshow(cv2.cvtColor(image,cv2.COLOR_BGR2RGB))
#     plt.title('Imagen original',fontsize=7)
#     plt.axis("off")
    
    # Convertir la imagen a espacio de color HSV
    hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    
#     plt.subplot(1,5,2)
#     plt.imshow(hsv_image)
#     plt.title('Imagen en espacio de color HSV',fontsize=7)
#     plt.axis("off")

    # Extraer el componente S de la imagen
    s_component = hsv_image[:, :, 1]

    # Aplicar un umbral para convertir el componente S en binario
    _, binary_image = cv2.threshold(s_component, 0.28 * 255, 255, cv2.THRESH_BINARY)

    # Crear una máscara con el fondo negro
    mask = cv2.merge((binary_image, binary_image, binary_image))
    
#     plt.subplot(1,5,3)
#     plt.imshow(mask)
#     plt.title('Máscara con fondo negro',fontsize=7)
#     plt.axis("off")

    # Extraer los componentes R, G, B de la imagen original
    r_component, g_component, b_component = cv2.split(image)

    # Aplicar la máscara a cada componente individualmente
    r_masked = cv2.bitwise_and(r_component, mask[:, :, 0])
    g_masked = cv2.bitwise_and(g_component, mask[:, :, 1])
    b_masked = cv2.bitwise_and(b_component, mask[:, :, 2])

    # Combinar los componentes enmascarados en una nueva imagen
    masked_image = cv2.merge((r_masked, g_masked, b_masked))
    
#     plt.subplot(1,5,4)
#     plt.imshow(cv2.cvtColor(masked_image,cv2.COLOR_BGR2RGB))
#     plt.title('Imagen sin fondo',fontsize=7)
#     plt.axis("off")    
    
    # Aplicar un filtro de media para reducir el ruido
    filtered_image = cv2.boxFilter(masked_image, -1, (5, 5))
    
#     plt.subplot(1,5,5)
#     plt.imshow(cv2.cvtColor(filtered_image,cv2.COLOR_BGR2RGB))
#     plt.title('Imagen sin ruido',fontsize=7)
#     plt.axis("off")    
#     plt.show()

    return filtered_image

# Resultados 
# image_path = "./data/enfermas/bacterial_leaf_blight (121).JPG"
# image = cv2.imread(image_path)
# pre_img= preprocess_image(image)

<h1 style="color:RGB(0,0,255)"> 5. Segmentación de imagen</h1>

In [88]:
# plt.figure(figsize=(13,5))

def segment_image(filtered_image):
    
#     plt.subplot(1,5,1)
#     plt.imshow(cv2.cvtColor(filtered_image,cv2.COLOR_BGR2RGB))
#     plt.title('Imagen Preprocesada',fontsize=7)
#     plt.axis("off")

    # Convertir la imagen sin fondo al espacio de color HSV
    hsv_image = cv2.cvtColor(filtered_image, cv2.COLOR_BGR2HSV)

    # Extraer el componente Hue de la imagen en el espacio de color HSV
    hue_component = hsv_image[:, :, 0]

    # Obtener las dimensiones de la imagen del componente Hue
    rows, cols = hue_component.shape

    # Transformar la imagen del componente Hue en un vector
    hue_vector = hue_component.reshape(-1)

    # Aplicar K-means en el componente Hue con #clusters = 3
    kmeans = KMeans(n_clusters=3, n_init=10, random_state=0)
    kmeans.fit(hue_vector.reshape(-1, 1))

    # Etiquetar cada píxel en la imagen de fondo eliminada usando el resultado de K-means
    segmented_image = kmeans.labels_.reshape(rows, cols)

    # Crear tres imágenes vacías de la misma dimensión para almacenar el resultado de la agrupación
    cluster1_image = np.zeros((rows, cols, 3), dtype=np.uint8)
    cluster2_image = np.zeros((rows, cols, 3), dtype=np.uint8)
    cluster3_image = np.zeros((rows, cols, 3), dtype=np.uint8)

    # Asignar cada píxel a la imagen respectiva en función de la detección de etiqueta/índice asignada
    
   # Obtener los colores originales de la imagen de entrada
    colors = filtered_image.reshape(-1, 3)

    # Asignar los colores originales a cada píxel en las imágenes segmentadas
    for i in range(rows):
        for j in range(cols):
            if segmented_image[i, j] == 0:
                cluster1_image[i, j] = colors[i * cols + j]
            elif segmented_image[i, j] == 1:
                cluster2_image[i, j] = colors[i * cols + j]
            elif segmented_image[i, j] == 2:
                cluster3_image[i, j] = colors[i * cols + j]
                               
#     plt.subplot(1,5,2)
#     plt.imshow(cv2.cvtColor(cluster1_image,cv2.COLOR_BGR2RGB))
#     plt.title('Cluster 1: Fondo',fontsize=7)
#     plt.axis("off")
    
#     plt.subplot(1,5,3)
#     plt.imshow(cv2.cvtColor(cluster2_image,cv2.COLOR_BGR2RGB))
#     plt.title('Cluster 2: Parte Sana',fontsize=7)
#     plt.axis("off")
    
#     plt.subplot(1,5,4)
#     plt.imshow(cv2.cvtColor(cluster3_image,cv2.COLOR_BGR2RGB))
#     plt.title('Cluster 3: Parte Enferma',fontsize=7)
#     plt.axis("off")

    enferma_image= cluster3_image
    # Convertir la imagen de la parte enferma al espacio de color HSV
    hsv_enferma = cv2.cvtColor(enferma_image, cv2.COLOR_BGR2HSV)

    # Definir los valores mínimo y máximo para el rango de color verde
    min_green = 17.2 / 360   # Convertir a escala de 0 a 1
    max_green = 45 / 360     # Convertir a escala de 0 a 1

    # Crear una máscara binaria basada en el rango de color verde para la parte enferma
    mask = cv2.inRange(hsv_enferma, (min_green, 0, 0), (max_green, 255, 255))

    # Aplicar la máscara a la imagen de la parte enferma para eliminar la porción verde
    enferma_sin_verde = cv2.bitwise_and(enferma_image, enferma_image, mask=cv2.bitwise_not(mask))

#     plt.subplot(1,5,5)
#     plt.imshow(cv2.cvtColor(enferma_sin_verde,cv2.COLOR_BGR2RGB))
#     plt.title('Sin Porción Verde',fontsize=7)
#     plt.axis("off") 
#     plt.show()     
    
    return cluster1_image, cluster2_image, enferma_sin_verde

# enferma_sin_verde = segment_image(pre_img)

<h1 style="color:RGB(0,0,255)"> 6. Extracción de Características</h1>

In [74]:
def extract_features(enferma_image):
    
    # Convertir la imagen a escala de grises
    enferma_gray = cv2.cvtColor(enferma_image, cv2.COLOR_BGR2GRAY)

    # Extraer características de color
    r_mean = np.mean(enferma_image[:, :, 2])
    g_mean = np.mean(enferma_image[:, :, 1])
    b_mean = np.mean(enferma_image[:, :, 0])
    r_std = np.std(enferma_image[:, :, 2])
    g_std = np.std(enferma_image[:, :, 1])
    b_std = np.std(enferma_image[:, :, 0])

    # Extraer características de forma
    enferma_mask = cv2.threshold(enferma_gray, 0.28, 1, cv2.THRESH_BINARY)[1]
    total_area = np.sum(enferma_mask)
    labeled_image = label(enferma_mask)
    num_spots = len(np.unique(labeled_image)) - 1

    # Extraer características de textura utilizando mahotas
    mahotas_glcm = texture.haralick(enferma_gray)
    mahotas_contrast = mahotas_glcm.mean(axis=0)[2]
    mahotas_correlation = mahotas_glcm.mean(axis=0)[4]
    mahotas_energy = mahotas_glcm.mean(axis=0)[8]
    mahotas_homogeneity = mahotas_glcm.mean(axis=0)[1]

    # Crear vector de características
    features85 = [
        r_mean, g_mean, b_mean, r_std, g_std, b_std,
        total_area, num_spots, 
        mahotas_contrast, mahotas_correlation, mahotas_energy, mahotas_homogeneity
    ]

    return features85

# extract_features(enferma_sin_verde)

<h1 style="color:RGB(0,0,255)"> 7. Clasificación de imágenes</h1>

<h2 style="color:RGB(0,0,0)"> 7.1 Agregar Características y etiquetas a cada clase</h2>

In [89]:
# las características extraídas para múltiples imágenes
# Guarda las características en una lista
features = []

# Guarda las etiquetas correspondientes a las características en una lista
labels = []

# Agrega las características y etiquetas de cada imagen a las listas
# Clase 1 - Imágenes
for img in enfermas_images:
    preprocessed_image = preprocess_image(img)
    _,_,segmented_image = segment_image(preprocessed_image)
    r_features = extract_features(segmented_image)
    # Agregar las características a la lista features
    features.append(r_features)
    labels.append(0)  # Etiqueta correspondiente a la clase de la imagen

# Clase 2 - Imágenes
for img in sanas_images:
    preprocessed_image = preprocess_image(img)
    _,_,segmented_image = segment_image(preprocessed_image)
    r_features = extract_features(segmented_image)
    # Agregar las características a la lista features
    features.append(r_features)
    labels.append(1)  # Etiqueta correspondiente a la clase de la imagen

# Determinar la longitud máxima de las características
max_length = max(len(f) for f in features)

# Rellenar o truncar las características para que tengan la misma longitud
for i in range(len(features)):
    if len(features[i]) < max_length:
        features[i] = features[i] + [0] * (max_length - len(features[i]))
    else:
        features[i] = features[i][:max_length]

<h2 style="color:RGB(0,0,0)"> 7.2 Clasificador SVM </h2>

In [96]:
# Convertir las listas de características y etiquetas en arreglos de NumPy
X = np.array(features)
y = np.array(labels)

# Divide los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)

# Asegurarse de que las características sean de tipo float si es necesario
X_train = X_train.astype(float)
X_test = X_test.astype(float)

# Crea un clasificador SVM para clasificación binaria

# clf = svm.SVC(C=1, kernel="linear", probability=True)
clf = svm.SVC(C=0.1, gamma='scale', kernel='linear', probability=True)

# Entrena el clasificador SVM utilizando los datos de entrenamiento
clf.fit(X_train, y_train)

#Guardar clasificador entrenado
with open('./SVC_model.pkl', 'wb') as f:
    pickle.dump(clf, f)

# Realiza predicciones en los datos de prueba
predictions = clf.predict(X_test)

# Evalúa el desempeño del clasificador
accuracy = clf.score(X_test, y_test)

# Imprime las predicciones y la precisión del clasificador
print("Predicciones:", predictions)
print("Precisión:", accuracy)

Predicciones: [1 1 1 1 1 0 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 0 1 0 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
Precisión: 0.9607843137254902


<h2 style="color:RGB(0,0,0)"> 7.2.1 Búsqueda de rejilla con validación cruzada </h2>

In [91]:
# Convertir las listas de características y etiquetas en arreglos de NumPy
X = np.array(features)
y = np.array(labels)

# Divide los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Asegurarse de que las características sean de tipo float si es necesario
X_train = X_train.astype(float)
X_test = X_test.astype(float)

# Crea un clasificador SVM para clasificación binaria
clf = svm.SVC()

# Define la cuadrícula de valores para los hiperparámetros a probar
param_grid = {
    'C': [0.1, 1, 10, 50, 100],
    'kernel': ['linear', 'rbf'],
    'gamma': ['scale', 'auto']
}

# Crea el objeto GridSearchCV con validación cruzada
grid_search = GridSearchCV(clf, param_grid, cv=5, scoring='accuracy')

# Realiza la búsqueda de hiperparámetros utilizando los datos de entrenamiento
grid_search.fit(X_train, y_train)

# Obtiene la mejor combinación de hiperparámetros y la mejor métrica de evaluación
best_params = grid_search.best_params_
best_score = grid_search.best_score_

# Imprime los resultados
print("Mejores hiperparámetros:", best_params)
print("Mejor precisión:", best_score)

Mejores hiperparámetros: {'C': 0.1, 'gamma': 'scale', 'kernel': 'linear'}
Mejor precisión: 0.97


<h2 style="color:RGB(0,0,0)"> 7.2.2 Matriz de Confusión </h2>

In [97]:
with open('./SVC_model.pkl', 'rb') as f:
    clf= pickle.load(f)

# Realiza predicciones en los datos de prueba
predictions = clf.predict(X_test)

# Obtén la matriz de confusión
confusion_mat = confusion_matrix(y_test, predictions)
print("Matriz de confusión:")
print(confusion_mat)

# Calcula la precisión, recall y F1-score
classification_rep = classification_report(y_test, predictions)
print("Reporte de clasificación:")
print(classification_rep)

Matriz de confusión:
[[ 2  0]
 [ 2 47]]
Reporte de clasificación:
              precision    recall  f1-score   support

           0       0.50      1.00      0.67         2
           1       1.00      0.96      0.98        49

    accuracy                           0.96        51
   macro avg       0.75      0.98      0.82        51
weighted avg       0.98      0.96      0.97        51



<h1 style="color:RGB(0,0,255)"> 8. Identificación de la Plaga</h1>

In [83]:
# Supongamos que ya entrenado el modelo y el clasificador SVM (clf) entrenado y listo para usar
# Cargar la imagen de prueba
desired_size = (512, 512)

with open('./SVC_model.pkl', 'rb') as f:
    clf= pickle.load(f)
image_path = "./data/enfermas/bacterial_leaf_blight (121).JPG"
image = cv2.imread(image_path)
image = cv2.resize(image, desired_size)

# Preprocesar la imagen
preprocessed_image = preprocess_image(image)

# Segmentar la imagen
_,_,segmented_image = segment_image(preprocessed_image)

# Extraer características de la imagen
features = extract_features(segmented_image)

# Convertir las características en un arreglo 2D de NumPy
X = np.array([features])

# Realizar la predicción utilizando el modelo entrenado
prediction = clf.predict(X)    

# Imprimir la clase predicha
print("Clase predicha:", prediction)

# Suponiendo que ya tienes tu clasificador entrenado y lo has llamado clf
# Suponiendo también que tienes tus datos de prueba en X_test y las etiquetas verdaderas en y_true

# Obtener las probabilidades de clasificación para cada clase
probs = clf.predict_proba(X)

# Obtener las predicciones de clase
predictions = clf.predict(X)

# Imprimir los resultados
for i in range(len(X)):
    print(f"Muestra {i + 1}:")
    print(f"Probabilidad clase sana: {probs[i][1]}")
    print(f"Probabilidad clase enferma: {probs[i][0]}")
    print(f"Predicción: {'sana' if predictions[i] == 1 else 'enferma'}")
    print()
    print(probs[i][1]+probs[i][0])  

Clase predicha: [0]
Muestra 1:
Probabilidad clase sana: 4.44942915178412e-05
Probabilidad clase enferma: 0.9999555057084821
Predicción: enferma

1.0


<h1 style="color:RGB(0,0,255)"> 9. Interfaz Final </h1>

In [99]:
import tkinter as tk
from tkinter import filedialog
from PIL import ImageTk, Image
import pickle
import numpy as np
import cv2

desired_size = (512, 512)

# Función para redimensionar la imagen a 128x128
def resize_image(image):
    return cv2.resize(image, (256, 256), interpolation=cv2.INTER_AREA)

# Cargar el modelo SVM previamente entrenado
with open('./SVC_model.pkl', 'rb') as f:
    clf = pickle.load(f)

# Función para cargar y mostrar la imagen seleccionada
test_img = []
def load_image():
    filepath = filedialog.askopenfilename(initialdir="./", title="Seleccione una imagen", filetypes=(("Archivos de imagen", "*.jpg;*.jpeg;*.png"), ("Todos los archivos", "*.*")))
    if filepath:       
        img = Image.open(filepath)
        img1 = img.resize((256, 256), Image.LANCZOS)
        img_tk = ImageTk.PhotoImage(img1)
        #------Imagen original
        image_labela.configure(image=img_tk)
        image_labela.image = img_tk

        # Realizar la predicción en la imagen cargada
        img_array = np.array(img)
        image = cv2.resize(img_array, desired_size)
        
        # Preprocesar la imagen
        preprocessed_image = preprocess_image(image)         
        # Redimensionar la imagen a 128x128
        resized_image1 = resize_image(preprocessed_image)        
        # Convertir la imagen de OpenCV a formato compatible con Tkinter
        img_tk1 = ImageTk.PhotoImage(Image.fromarray(resized_image1))
       
        # Segmentar la imagen
        segmented_1,segmented_2,segmented_image = segment_image(preprocessed_image)
        # Redimensionar la imagen a 128x128----------------------------------------
        resized_image2 = resize_image(segmented_image)  
        # Convertir la imagen de OpenCV a formato compatible con Tkinter
        img_tk2 = ImageTk.PhotoImage(Image.fromarray(resized_image2))                
        #eliminacion de fondo - preprocesamiento
        image_label1.config(image=img_tk1)
        image_label1.image = img_tk1        

        #segmented_image:enfermo -segmentar
        image_label2.config(image=img_tk2)
        image_label2.image = img_tk2      
        
        # Extraer características de la imagen
        features = extract_features(segmented_image)

        # Convertir las características en un arreglo 2D de NumPy
        X = np.array([features])

        # Realizar la predicción utilizando el modelo entrenado
        prediction = clf.predict(X)    

        # Calcula probabilidad de cada clase
        probs = clf.predict_proba(X)

        # # Imprimir la clase predicha y los resultados
        for i in range(len(X)):
            print(f"Muestra {i + 1}:")
            print(f"Probabilidad clase sana: {probs[i][1]}")
            print(f"Probabilidad clase enferma: {probs[i][0]}")
            
        prediction_label.configure(text="Predicción: {}".format('SANA' if prediction[i] == 1 else 'ENFERMA'))
#         additional_label.configure(text="Probabilidades:")
        additional_label1.configure(text="Enferma: {:.6f}".format(probs[i][0]))
        additional_label2.configure(text="Sana: {:.6f}".format(probs[i][1]))
               

# Crear la ventana principal
window = tk.Tk()
window.title("PROYECTO FINAL - PEVICO")
#window.geometry("1000x800")

# Obtener el tamaño de la pantalla
screen_width = window.winfo_screenwidth()
screen_height = window.winfo_screenheight()

# Calcular las coordenadas x e y para posicionar la ventana en el centro
window_width = 1000  # Ancho deseado de la ventana
window_height = 650  # Alto deseado de la ventana
x = (screen_width - window_width) // 2
y = (screen_height - window_height) // 2

# Establecer la posición de la ventana en el centro
window.geometry(f"{window_width}x{window_height}+{x}+{y}")

# Cambiar el color de fondo (código hexadecimal)
window.configure(bg="#FFEEDB")

#Título
original_label0 = tk.Label(window, text="Identificador del Tizón bacteriano de la hoja del arroz 'Bacterial leaf blight of rice'", font=("Verdana", 15, "bold"), fg='#06132F',bg="#FFEEDB")
original_label0.place(relx=0.5, rely=0.05, anchor=tk.CENTER)
#botnn
load_button = tk.Button(window, text="Cargar imagen", command=load_image, font=("Verdana", 14), fg="white", bg="blue")
load_button.pack(pady=65, padx=35, anchor="center")  # Ajustar el espaciado vertical (5) y horizontal (20)

#------------------------------------------------------------------
original_label0 = tk.Label(window, text="Imagen Original                    Preprocesamiento                    Segmentación", font=("Verdana", 15, "bold"), fg='#06132F',bg="#FFEEDB")
original_label0.place(relx=0.5, rely=0.20, anchor=tk.CENTER)

# Contenedor para las imágenes
images_container = tk.Frame(window)
images_container.pack(pady=10)

# Crear las etiquetas para las imágenes
image_labela = tk.Label(images_container)
image_label1 = tk.Label(images_container)
image_label2 = tk.Label(images_container)

# Empaquetar las etiquetas de imagen una al lado de la otra
image_labela.pack(side=tk.LEFT, padx=40)
image_label1.pack(side=tk.LEFT, padx=40)
image_label2.pack(side=tk.LEFT, padx=40)
#------------------------------------------------------------------
# Etiqueta para mostrar la imagen cargada
original_label0 = tk.Label(window, text="Resultados", font=("Verdana", 15, "bold"), fg='black',bg="#FFEEDB")
original_label0.pack(pady=10, padx=35, anchor="center")  # Espaciado y anclaje a la izquierda (Oeste)

#Etiqueta para mostrar el resultado de la predicción
prediction_label = tk.Label(window, text="Predicción: " ,font=("Verdana", 18, "bold"), fg="blue", bg="#FFEEDB")
prediction_label.pack(pady=10)

# additional_label = tk.Label(window, text="Probabilidades:",font=("Verdana", 10, "bold"), fg='#06132F',bg="#FFEEDB")
# additional_label.pack(pady=5)

additional_label1 = tk.Label(window, text="Enferma:",font=("Verdana", 13, "bold"), fg='white',bg="red")
additional_label1.pack(pady=5)

additional_label2 = tk.Label(window, text="Sana:",font=("Verdana", 13, "bold"), fg='white',bg="green")
additional_label2.pack(pady=5)

# Ejecutar la aplicación
window.mainloop()

Muestra 1:
Probabilidad clase sana: 0.14505743531196585
Probabilidad clase enferma: 0.8549425646880341
